In [4]:
import catboost
import pandas as pd
import numpy as np
from typing import List
import pandas as pd
from scipy import stats
import datetime
import rpy2
from functools import reduce
from catboost import CatBoostRegressor
import time
import seaborn as sns
import ast
import numpy as np
from dateutil import parser
from kshape.core import kshape, zscore
from tslearn.clustering import KShape
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import catboost

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Path fo import files
#input data
import_data = "import/ecs_series_1_uid_encoded(1).csv"
#generated data
transformed_analyzed_data = 'export/alibaba/alibaba_initial_time_series_2000.csv'
monetary_time_series = "export/alibaba/alibaba_monetary_2000.csv"
frequency_time_series = "export/alibaba/alibaba_frequency_2000.csv"
recency_time_series = "export/alibaba/alibaba_recency_2000.csv"
cluster_centroids_time_series = "export/alibaba/alibaba_cluster_centroids_2000.csv"
#prediction
prediction_path = "export/alibaba/alibaba_prediction.csv"

## 0. Preprocessing Alibaba Data

In [6]:
df = pd.read_csv(import_data)
datetime_object = datetime.datetime.strptime("9/27/18 16:00", "%m/%d/%y %H:%M")
df['Date'] = df["ds"].apply(lambda x:datetime.datetime.strptime(x, "%m/%d/%y %H:%M"))
df = df.drop(columns = ['ds','region_no_factor','iz_no_factor','product_factor','instance_type_name_factor','instance_type_family_factor'])
df = df.groupby(by=['uid','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'uid',values = 'vcpu_net_delead')

## 0.1 Taking random 2000 users

In [7]:
df_test = df.sample(2000)
df_test.to_csv(transformed_analyzed_data)

## 1.0 RFM-analysis 

In [8]:
df_recency =  pd.DataFrame(columns = df_test.columns)
df_monetary =  pd.DataFrame(columns = df_test.columns)
df_frequency = pd.DataFrame(columns = df_test.columns)

In [ ]:
"""
The dataframe should have such columns: id of consumer, other columns as a time series
"""
for index, row in df_test.iterrows():
    arr = []
    arr = list(row)
    rec_arr = []
    mon_arr = []
    freq_arr = []
    start_date = row.index[0]
    last_purchase = np.nan
    first_purchase = -1 #flag
    total_purchases = 0
    #RECENCY Customer did a explicit activity, such as renting cores or releasing cores.
    for j, entry in enumerate(arr):
        if (np.isnan(entry)):            
            if (first_purchase==-1):
                rec_arr.append(row.index[j]-start_date)
            else:
                rec_arr.append(row.index[j]-last_purchase)
        else:
            #not nan
            total_purchases+=1
            if (first_purchase==-1):
                first_purchase = j
            rec_arr.append(0)
            last_purchase = row.index[j]
        #FREQUENCY it can be a cumulative average, the number of events divided by the number of days that have passed
        #since first event
        freq_arr.append(total_purchases/(j+1-first_purchase))
    df_recency.loc[index] = rec_arr
    df_frequency.loc[index]=freq_arr
    #MONETARY  Moving average monetary value
    current_servers = 0
    total_days_of_purchases = 0 
    first_purchase = -1#flag
    for j,entry in enumerate(arr):
        if (np.isnan(entry)):
            total_days_of_purchases+=current_servers
        else:
            current_servers+=entry
            total_days_of_purchases+=current_servers
            if (first_purchase==-1):
                first_purchase = j
        mon_arr.append(total_days_of_purchases/(j+1-first_purchase))
    df_monetary.loc[index] = mon_arr
df_recency=df_recency.applymap( lambda x:x if (type(x)==int) else x.days)

## 2. Clustering

In [ ]:
ksh_monetary = KShape(n_clusters=4)
df_monetary['label'] = ksh_monetary.fit(df_monetary.values)
ksh_frequency = KShape(n_clusters=4)
df_frequency['label'] = ksh_frequency.fit(df_frequency.values)
ksh_recency = KShape(n_clusters=4)
df_recency['label'] = ksh_recency.fit(df_recency.values)

In [ ]:
df_monetary['label'] =ksh_monetary.labels_
df_frequency['label'] =ksh_frequency.labels_
df_recency['label'] =ksh_recency.labels_

In [ ]:
df_centroids = pd.DataFrame(columns = df_recency.drop(columns = 'label').columns)
for i in range(ksh_recency.cluster_centers_.shape[0]):
    df_centroids.loc['Centroid_for_recency_'+str(i)]= ksh_recency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_frequency_'+str(i)]= ksh_frequency.cluster_centers_[i].ravel()
    df_centroids.loc['Centroid_for_monetary_'+str(i)]= ksh_monetary.cluster_centers_[i].ravel()

In [ ]:
df_monetary.to_csv(monetary_time_series)
df_frequency.to_csv(frequency_time_series)
df_recency.to_csv(recency_time_series)
df_centroids.to_csv(cluster_centroids_time_series)

## 3. Preprocessing for prediction

In [2]:
df = pd.read_csv(transformed_analyzed_data)
df.head()

,uid,2017-01-01 16:00:00,2017-01-02 16:00:00,2017-01-03 16:00:00,2017-01-04 16:00:00,2017-01-05 16:00:00,2017-01-06 16:00:00,2017-01-07 16:00:00,2017-01-08 16:00:00,2017-01-09 16:00:00,...,2018-11-16 16:00:00,2018-11-17 16:00:00,2018-11-18 16:00:00,2018-11-19 16:00:00,2018-11-20 16:00:00,2018-11-21 16:00:00,2018-11-22 16:00:00,2018-11-23 16:00:00,2018-11-24 16:00:00,2018-11-25 16:00:00
0,22864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,84126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,137874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = pd.melt(df,id_vars = ['uid'],var_name ='timestamp' ,value_name='value')
df.head()

,uid,timestamp,value
0,22864,2017-01-01 16:00:00,NaN
1,84126,2017-01-01 16:00:00,NaN
2,36465,2017-01-01 16:00:00,NaN
3,49628,2017-01-01 16:00:00,NaN
4,137874,2017-01-01 16:00:00,NaN


In [4]:
df_freq = pd.read_csv(frequency_time_series).loc[:,['Unnamed: 0','label']]
df_monet = pd.read_csv(monetary_time_series).loc[:,['Unnamed: 0','label']]
df_recency = pd.read_csv(recency_time_series).loc[:,['Unnamed: 0','label']]

In [5]:
df_monet.head()

,Unnamed: 0,label
0,22864,2
1,84126,2
2,36465,2
3,49628,2
4,137874,0


In [6]:
df_combined = df.merge(df_recency,left_on = 'uid',right_on='Unnamed: 0') \
    .merge(df_freq,left_on = 'uid',right_on='Unnamed: 0') \
    .merge(df_monet,left_on = 'uid',right_on='Unnamed: 0') \
    .drop(columns =['Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0'])
df_combined.columns = ['uid','timestamp','value','recency','frequency','monetary']
df_combined.sample(5)

,uid,timestamp,value,recency,frequency,monetary
1198593,105414,2018-02-02 16:00:00,NaN,1,0,2
1111659,68403,2017-03-30 16:00:00,NaN,2,0,2
344350,67411,2018-09-16 16:00:00,NaN,2,0,3
863599,36412,2017-05-03 16:00:00,NaN,1,0,2
542554,13695,2018-09-22 16:00:00,NaN,3,0,2


In [7]:
#For CDNow
df_combined['timestamp'] = df_combined['timestamp'].apply(lambda x:datetime.datetime.strptime(x[:10], "%Y-%m-%d"))
df_combined.sample(2)

,uid,timestamp,value,recency,frequency,monetary
1309285,149434,2017-07-29,NaN,1,0,2
1228332,136454,2017-12-04,NaN,1,0,0


In [8]:
df_combined = df_combined.fillna(0)

In [9]:
# number of months
test_period = 5
current_date = df_combined.timestamp.max()
df_test = df_combined[df_combined.timestamp >= current_date - pd.DateOffset(months = test_period )]
df_train = df_combined[df_combined.timestamp < current_date - pd.DateOffset(months = test_period )]

In [10]:
df_test.shape[0]/df_train.shape[0]

0.2857142857142857

In [11]:
y_train = df_train['value']
y_test = df_test['value']

X_train = df_train.drop('value', axis = 1)
X_test = df_test.drop('value', axis = 1)

## 4. Prediction

In [14]:
start_time = time.time()

cv = KFold(n_splits = 3, shuffle = True, random_state = 42)  

model = CatBoostRegressor(verbose = False, 
                           random_state=42)

param_grid = {
              'n_estimators': [200, 300],
              'max_depth': [3, 4], 
              'learning_rate': [0.03, 0.05, 0.07] 
             }

gs_cb = GridSearchCV(model, 
                  param_grid,
                  cv = cv,
                  scoring = 'neg_mean_squared_error')

gs_cb.fit(X_train, y_train)

#train_RMSE = np.sqrt(-gs_cb.cv_results_['mean_train_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()
#train_std = np.sqrt(gs_cb.cv_results_['std_train_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()

cv_RMSE = np.sqrt(-gs_cb.cv_results_['mean_test_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()
cv_std = np.sqrt(gs_cb.cv_results_['std_test_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()

test_RMSE = np.sqrt(mean_squared_error(gs_cb.predict(X_test), y_test))

#print('train RMSE: {} +/- {}'.format(np.round(train_RMSE, 2), np.round(train_std, 2)))
print('CV RMSE: {} +/- {}'.format(np.round(cv_RMSE, 2), np.round(cv_std, 2)))
print('test RMSE: {}'.format(np.round(test_RMSE, 2)))
print(gs_cb.best_params_)
print('time :{} min'.format(np.round((time.time() - start_time)/60, 2))) 

filename = 'CatBoost_alibaba.sav'
pickle.dump(gs_cb, open(filename, 'wb'))

CV RMSE: 0.02 +/- 0.02
test RMSE: 0.05
{'learning_rate': 0.03, 'max_depth': 3, 'n_estimators': 200}
time :12.18 min


In [15]:
result = X_test
result['predicted_value']=gs_cb.predict(X_test)
result['true_value']=y_test


result.to_csv(prediction_path)